In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
import pandas as pd
import PIL
import scipy.sparse as sp
import resource
import time
from PIL import Image
from PIL import ImageFile
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import json, pickle

In [ ]:
import re
import pandas
import numpy as np
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.text import Tokenizer
from nltk.corpus import names
from nltk.stem import WordNetLemmatizer

In [ ]:
from dataloader import ReviewCoverLoader

In [ ]:
import warnings 
warnings.filterwarnings(action='ignore')

In [ ]:
def set_env():
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.75
    session = tf.Session(config=config)
    session
set_env()

In [ ]:
InputSize_wh = 32
batch_size = 32

soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (hard, hard))

In [ ]:
total_result = pd.DataFrame()
accuracy_avg = 0

In [ ]:
test_file_names = ['test_number', 'pad_Test', 'test_num2resizedImg', 'test_obj', 'test_label']
test_number,pad_Test,test_num2resizedImg,test_obj,test_label = [], [], [], [], []
test_files = [test_number,pad_Test,test_num2resizedImg,test_obj,test_label]

In [ ]:
with open(f"test_data/preprocessed_data/test_number.h5","rb") as f:test_number = pickle.load(f)
with open(f"test_data/preprocessed_data/pad_Test.h5","rb") as f:pad_Test = pickle.load(f)
with open(f"test_data/preprocessed_data/test_num2resizedImg.h5","rb") as f:test_num2resizedImg = pickle.load(f)
with open(f"test_data/preprocessed_data/test_obj.h5","rb") as f:test_obj = pickle.load(f)
with open(f"test_data/preprocessed_data/test_label.h5","rb") as f:test_label = pickle.load(f)

In [ ]:
for ind, name in enumerate(test_file_names):
    with open(f"test_data/preprocessed_data/{name}.h5","rb") as f:
        test_files[ind] = pickle.load(f)

In [ ]:
model1=tf.keras.models.load_model("main_model.h5")

In [ ]:
fused_predict = model1.predict_generator(ReviewCoverLoader(
    test_number,pad_Test,test_num2resizedImg,test_obj,test_label, InputSize_wh, batch_size,shuffle=False))
report_score = classification_report(test_label[:len(fused_predict)], fused_predict.argmax(axis=1),digits=4, output_dict=True)
accuracy_avg+=report_score["accuracy"]
report_score = pd.DataFrame(report_score)

if(ind==0):total_result = report_score
else:total_result += report_score

print('\n',report_score.T)
print()